In [1]:
import numpy as np
import threading
import queue
#testArray = np.array([[1,2,3,4],[2,3,4,1],[3,4,1,2],[4,1,2,3],[3,4,1,2],[4,1,2,3]])
testArray = np.array([[1, 2], [2, 1]])
np.shape(testArray)

(2, 2)

In [2]:
def simpleNMF(A, k, i):
    m, n = np.shape(A)
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    for _ in range(i):
        W = W * (A @ H.T) / (W @ H @ H.T)
        H = H * (W.T @ A) / (W.T @ W @ H)
    return W @ H

simpleNMF(testArray, 2, 1000)

array([[1., 2.],
       [2., 1.]])

In [3]:
def thread_function_W(A, W, H, q, i):
    q.put((i, W * (A @ H.T) / (W @ H @ H.T)))

def thread_function_H(A, W, H, q, i):
    q.put((i, H * (W.T @ A) / (W.T @ W @ H)))

class matrixHolder():
    def __init__(self):
        self.value = None
    
    def get(self):
        return self.value

    def set(self, newValue):
        self.value = newValue


def naiveParallelNMF(A, k, p, numIter):
    m, n = np.shape(A)
    if m % p > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % p > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    print(f"initial: {W@H}")
    As1 = [A[int(i*m/p):int((i+1)*m/p), :] for i in range(p)]
    As2 = [A[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)]
    
    for _ in range(numIter):
        Ws = [W[int(i*m/p):int((i+1)*m/p), :] for i in range(p)] # chop up W
        w_threads = []
        w_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_W, args = (As1[j], Ws[j], H, w_queue, j)) # each thread updates one section
            newThread.start()
            w_threads.append(newThread)
        for thread in w_threads: # wait for all threads to complete
            thread.join()
        while not w_queue.empty(): # reconstitute and update W
            i, v = w_queue.get()
            Ws[i] = v
        W = np.concatenate(Ws, 0)

        Hs = [H[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)] # same procedure for H
        h_threads = []
        h_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_H, args = (As2[j], W, Hs[j], h_queue, j))
            newThread.start()
            h_threads.append(newThread)
        for thread in h_threads:
            thread.join()
        while not h_queue.empty():
            i, v = h_queue.get()
            Hs[i] = v
        H = np.concatenate(Hs, 1)
    return W @ H
    
naiveParallelNMF(testArray, 2, 2, 10000)

initial: [[0.53708831 0.69200755]
 [0.11947232 0.1884871 ]]


array([[1., 2.],
       [2., 1.]])

In [4]:
def HPC_NMF(A, k, p1, p2, numIter):
    m, n = np.shape(A)
    if m % (p1*p2) > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % (p1*p2) > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)

    Hs = [[np.random.rand(k, int(n/(p1*p2))) for i in range(p)] for j in range(p2)]


    return A